In [ ]:
# Вначале необходимо установить используемые библиотеки.

In [1]:
!pip list -v

Package                                  Version              Location                                Installer
---------------------------------------- -------------------- --------------------------------------- ---------
absl-py                                  1.4.0                /usr/local/lib/python3.12/dist-packages uv
accelerate                               1.12.0               /usr/local/lib/python3.12/dist-packages uv
access                                   1.1.10.post3         /usr/local/lib/python3.12/dist-packages uv
affine                                   2.4.0                /usr/local/lib/python3.12/dist-packages uv
aiofiles                                 24.1.0               /usr/local/lib/python3.12/dist-packages uv
aiohappyeyeballs                         2.6.1                /usr/local/lib/python3.12/dist-packages uv
aiohttp                                  3.13.2               /usr/local/lib/python3.12/dist-packages uv
aiosignal                                

In [2]:
!pip install accelerate transformers

In [3]:
!pip install --upgrade langchain langchain-huggingface langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install langgraph

In [6]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=ad6684bf9f34bc7e903a167fb45572f7be613b2c7b7d8d968a32d45cc6a0e87d
  Stored in directory: /root/.cache/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia


In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain_huggingface import ChatHuggingFace
import torch

In [8]:
# Можно использовать разные модели. Чем больше размер модели, тем в среднем лучше результат.
# Также необходимо помнить что некоторые модели специально дообучаются для использования в качесвте агентов,
# их учат использовать созданные для них инструменты и следовать инструкциям.

# Указываем имя модели и загружаем токенизатор и модель
model_id = "Qwen/Qwen3-4B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    dtype=torch.float16,
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [9]:
# Создаем pipeline Hugging Face
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0.3,
    top_p=0.95,
    repetition_penalty=1.2
)

# На основе pipeline Hugging Face создаем langchain pipeline и langchain chat model
# И то и то можно использовать для построения агентной системы. Основное отличие — структура ввода и вывода.
# pipeline принимает и выдает текст, тогда как chat model работает с запросами в виде диалогов.
llm = HuggingFacePipeline(pipeline=pipe)
chat_model = ChatHuggingFace(llm=llm)

Device set to use cuda:0


In [10]:
prompt = "What do you know of the city of Penza, Russia?"
response = llm.invoke(prompt)
print(response)

What do you know of the city of Penza, Russia? What is its history and culture?

Penza (Пенза) is a city in southern Russia, located on the banks of the **Volga River**, approximately 120 kilometers (75 miles) south of Moscow. It is part of the **Penzenskaya Oblast** (Penzensk Region), which is one of the smaller federal subjects of Russia.

### History:
- **Ancient Times**: The area around Penza has been inhabited for thousands of years. Archaeological findings suggest that it was settled by various ancient tribes.
  
- **Medieval Period**: In the medieval period, the region became an important center due to its strategic location along trade routes connecting Europe and Asia. The first known settlement dates back to the early 9th century when it was mentioned as "Penzia" or "Penzhik."

- **Tsarist Era**: During the reign of Tsars like Ivan III and Ivan IV (Ivan the Terrible), the area came under Russian control. By the late 16th century, the town had grown into a significant administ

In [11]:
# Ответ chat model содержит специальные теги начала и окончания фразы, метки указывающие кому принадлежит текст
# user - пользовательский контент, assistant - текст модели
# Для размышляющих моделей в диалоге может присутствовать тег think помечающий внутренние размышления модели.
prompt = "Какие последние успехи на научном поприще в городе Пенза?"
response = chat_model.invoke(prompt)
print(response)

content='<|im_start|>user\nКакие последние успехи на научном поприще в городе Пенза?<|im_end|>\n<|im_start|>assistant\n<think>\nХорошо, пользователь спрашивает о последних успехах в области науки и технологий в городе Пензе. Нужно собрать информацию по этому вопросу.\n\nСначала вспомню, что Пенза — город в России, известный своей исторической значимостью и развитой индустрией. Возможно, есть какие-то университеты или исследовательские центры там. Например, Пензенский государственный университет (ПГУ) может быть важным элементом. Также стоит упомянуть Институт прикладной физики им. А.М. Прохорова, который находится в Пензе. Это один из крупных учреждений, занимающихся наукой.\n\nНадо проверить, были ли недавние достижения. Может быть, исследования в области информационных технологий, нанотехнологий или энергетики. Важно не перепутать с другими регионами, например, с Казани или Нижним Новгородом. \n\nТакже можно подумать про международные проекты или участие в космических программах. Нап

In [12]:
# Построим простейшую RAG (Retrieval-Augmented Generation или генерация, дополненная поиском) систему
# Такая система помогает LLM использовать информацию из внешних источников для формирования ответа, а не только собственные знания.

# В качестве источника дополнительной информации будем использовать простой текстовый документ
from langchain_community.document_loaders import TextLoader

# Загружаем документ (убедитесь что путь указан правильно, также возможны проблемы с кодировкой)
loader = TextLoader("Article.txt", encoding='cp1251')
documents = loader.load()

print(f"Loaded {len(documents)} document(s).")

Loaded 1 document(s).


In [13]:
# Для разных версий langchain может меняться формат импорта нужных компонент

# from langchain.text_splitter import CharacterTextSplitter
from langchain_text_splitters import CharacterTextSplitter

# Большие объемы информации обычно обрабатываются кусками - чанками.
# Это связано с ограничениями контекстного окна моделей.
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

print(f"Split into {len(texts)} chunks.")

Split into 23 chunks.


In [15]:
# Убедимся что информация загружена и нарезана, чанки содержат текст.
print(texts[19].page_content)

«Яндекс» продолжает повышать точность ответов, объём данных для обучения моделей и функционал. В этот раз получился один большой релиз: базовая клиентская версия, клиентская версия Pro, клиентская версия Lite, интеграция в Cloud для бизнеса.

Сбер

В апреле 2023 года «Сбер» представил GigaChat — нейросеть которая может одновременно генерировать и предложения, и картинки по текстовому описанию.

    В 2023 году была создана GigaChat - генеративная языковая модель, созданная Сбером. Она обучалась на огромном количестве текстовых данных и была разработана для помощи пользователям в решении различных задач, связанных с текстом.

    Волну искусственного интеллекта так же подхватил Сбер, начав разрабатывать свою нейросеть на основе модели ruGPT-3.5. Первым релизом так же была генеративная нейронка для работы с текстом.

    В марте 2023 года была интегрирована более продвинутая модель – ruGPT- 4.


In [16]:
К# Процесс RAG включает несколько этапов:
# Retrieval — поиск и извлечение фрагментов информации из базы знаний, которые наиболее релевантны запросу пользователя.
# Augmentation — дополнение запроса пользователя извлечённой информацией.
# Generation — генерация ответа языковой моделью с учётом дополнительной информации.
# Эти этапы еализованы в классе RetrievalQA
from langchain_classic.chains import RetrievalQA
from langchain_community.vectorstores import InMemoryVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings

# Сама информация в RAG системе хранится в векторном виде для облегчения поиска релевантных фрагментов.
# Для преобразования в векторный вид используюся Embedd-еры
embeddings = HuggingFaceEmbeddings()

# Преобразуем чанки в вектора, которые будут храниться в памяти.
# В реальной системе в памяти хранятся только наиболее часто используемы чанки,
# основной объем хранится в векторной базе данных.
# LangChain поддерживает различные векторные базы данных, например Chroma,
# Weaviate, Qdrant или база данных Яндекса YDB
vectorstore = InMemoryVectorStore.from_documents(texts, embeddings)

# retrievers сравнивают пользовательский запрос с векторизованными чанками (документами)
# и возвращают наиболее релевантные результаты.
# Для определения релевантности могут использоваться различные метрики, например, косинусная мера или совпадение ключевых слов.
retriever = vectorstore.as_retriever()


# Создадим цепочку формирования ответа на основе RAG
#
# Тип цепи "stuff" все данные от retriever добавляет в контекст запроса к языковой модели.
# Этот подход работает с небольшими фрагментами данных.
# При работе с большим количеством данных используются "refine" или "map_reduce", которые
# позволяют обрабатывать найденные документы каждый по отдельности уточняя и объединяя полученную из них информацию.
qa_chain = RetrievalQA.from_chain_type(
    chat_model,
    retriever=retriever,
    chain_type="stuff"
)

print("Question answering chain created.")


/tmp/ipython-input-3824565421.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
/tmp/ipython-input-3824565421.py:12: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Question answering chain created.


In [17]:
# Теперь можно формировать запросы, LLM будет изпользовать загруженные данные для ответа.
# Качество ответов зависит от многих факторов - от используемого Embedd-ера, метрики для поиска релевантных документов,
# поноты информации в самих документах и используемой LLM.
query = 'Кто В феврале 2023 года сообщил, что разрабатывает собственную версию генеративной нейросети ChatGPT?'
result = qa_chain.invoke({"query": query})
print(result['result'])

<|im_start|>system
Use the following pieces of context to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
30 ноября 2022 года был запущен ChatGPT - чат-бот с искусственным интеллектом, разработанный компанией OpenAI и способный работать в диалоговом режиме, поддерживающий запросы на естественных языках

    Это уже всем известный ChatGPT. Лишь написав одно предложение в чат, пользователь мог получить: простой программный алгоритм, рецепт, рекомендацию, идею, совет и многое другое.

    Исторический день, после которого все, кто позиционируют себя как IT гиганты, не могли не принять вызов рынка и не начать работу над своим аналогом.

    В марте 2023 года была интегрирована более продвинутая модель – GPT- 4.

    В данном релизе модель обучилась на существенно бОльшем объёме денных, научилась распознавать и работать с изображением. Качество ответов существенно улучшилось. Именно про GPT-4 я стал слы

In [ ]:
# Еще один спостоб построения систем на базе LLM - это агенты.
# Основное отличие агентов - это возможность планирования своих действий и активного взаимодействия
# со специально созданными для них инструментами или другими агентами.

In [18]:
# В langchain есть встроенные инструменты, предоставляющие агентам возможность
# делать запросы к поисковым движкам или внешним сервисам, выполнять команды в терминале и т.д.
# Создадим инструмент для доступа к Википедии, чтобы агент мог
# верифицировать свой ответ.

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1000)
wikitool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [19]:
print(wikitool.invoke({"query": "Верховцев"}))

Page: The Mystery of the Third Planet
Summary: The Mystery of the Third Planet (Russian: Тайна третьей планеты, Tayna tretyey planety, also translated as The Secret of the Third Planet) is a 1981 Soviet traditionally animated feature film directed by Roman Kachanov and produced by the Soyuzmultfilm studio in Moscow. It is based on a 1964 novella Alisa's Travel by Kir Bulychev, part of the Alisa Seleznyova children's science fiction book series.
The movie is considered a cult classic in Russia and was included in various lists of the best animated films and science fiction films. A shortened novelization of The Mystery of the Third Planet was written by Bulychev himself, with other adaptations including a diafilm and a number of video games. The 2009 feature film Alice's Birthday is considered a spiritual successor of the work.




In [20]:
# создаем новый tool для нашей ЛЛМ
from langchain.tools import tool
from datetime import datetime

@tool
def get_current_date() -> str:
    """Возвращает текущую дату в формате YYYY-MM-DD"""
    return datetime.now().strftime("%Y-%m-%d")

Обязательно добавляем новый тул в список

In [21]:
# Создадим список инструментов и агента с доступом к ним.
from langgraph.prebuilt import create_react_agent

tools = [wikitool,
         get_current_date]

agent_executor = create_react_agent(chat_model, tools)

/tmp/ipython-input-3244145801.py:7: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent_executor = create_react_agent(chat_model, tools)


In [22]:
# Ассистенты обрабатывают сообщения. У сообщений есть системная часть
# в которой описываеся его роль, инструкции
# и возможности использовать инструменты.

# Пользовательский запрос содержится в пользовательской части сообщения.
input_message = {
    "role": "user",
    "content": "Какая сейчас дата?"
}

result = agent_executor.invoke({"messages": [input_message]})
print(result["messages"][1])

content='<|im_start|>user\nКакая сейчас дата?<|im_end|>\n<|im_start|>assistant\n<think>\nХорошо, пользователь спрашивает: "Какая сейчас дата?" Нужно ответить на этот вопрос. Сначала я должен определиться с текущей датой.\n\nСейчас мне нужно проверить, какую дату показывает мой внутренний календарь или источник данных. Поскольку я не могу получить реальное время в режиме онлайн, я должен использовать последнюю информацию, которая была обновлена до 2023 года. \n\nПосле того как я уточню, что данные о времени и дне недели актуальны для октября 2023 года, можно приступить к составлению ответа. Важно указать точную дату и день недели, чтобы пользователь получил четкое представление о настоящем моменте.\n\nТакже стоит подумать, возможно ли, что пользователь хочет узнать конкретный формат даты (например, DD.MM.YYYY), но обычно достаточно просто назвать дату и день недели. Если есть необходимость, можно предложить дополнительные варианты форматирования.\n\nВ итоге, после всех рассуждений, я по

In [23]:
# Для удобства вывода используется специальный метод
for message in result["messages"]:
  message.pretty_print()

================================ Human Message =================================

Какая сейчас дата?
================================== Ai Message ==================================

<|im_start|>user
Какая сейчас дата?<|im_end|>
<|im_start|>assistant
<think>
Хорошо, пользователь спрашивает: "Какая сейчас дата?" Нужно ответить на этот вопрос. Сначала я должен определиться с текущей датой.

Сейчас мне нужно проверить, какую дату показывает мой внутренний календарь или источник данных. Поскольку я не могу получить реальное время в режиме онлайн, я должен использовать последнюю информацию, которая была обновлена до 2023 года. 

После того как я уточню, что данные о времени и дне недели актуальны для октября 2023 года, можно приступить к составлению ответа. Важно указать точную дату и день недели, чтобы пользователь получил четкое представление о настоящем моменте.

Также стоит подумать, возможно ли, что пользователь хочет узнать конкретный формат даты (например, DD.MM.YYYY), но обычно дост

In [24]:
#from langchain.agents import create_react_agent
from langchain.agents import create_agent
# Системный промпт можно создать самостоятельно.
my_prompt = """
Ты полезный ассистент.

Если вопрос требует:
- текущей даты или времени — ОБЯЗАТЕЛЬНО используй инструмент get_current_date
- проверки фактов — используй Wikipedia

Никогда не угадывай дату сам.
В финальном ответе явно укажи, какой инструмент был использован.
"""

agent_executor = create_agent(chat_model, tools, system_prompt=my_prompt)


In [25]:
input_message = {
    "role": "user",
    "content": "Who is Komlogorov?"
}

result = agent_executor.invoke({"messages": [input_message]})

for message in result["messages"]:
  message.pretty_print()

================================ Human Message =================================

Who is Komlogorov?
================================== Ai Message ==================================

<|im_start|>system

Ты полезный ассистент.

Если вопрос требует:
- текущей даты или времени — ОБЯЗАТЕЛЬНО используй инструмент get_current_date
- проверки фактов — используй Wikipedia

Никогда не угадывай дату сам.
В финальном ответе явно укажи, какой инструмент был использован.
<|im_end|>
<|im_start|>user
Who is Komlogorov?<|im_end|>
<|im_start|>assistant
<think>
Okay, the user is asking "Who is Komlogorov?" First, I need to figure out if this is a real person or maybe a misspelling. Let me check the spelling. "Komlogorov" doesn't ring a bell. Maybe it's supposed to be "Kolmogorov"? Yes, that sounds familiar. Alexander Kolmogorov is a well-known mathematician.

I should verify this using Wikipedia. Let me search for "Alexander Kolmogorov" on Wikipedia. The article mentions he was a Russian mathematician a

In [26]:
# Инструменты можно создавать свои.
# Но нужно тщательно формировать описание инструмента, т.к.
# агент сам решает когда его использовать именно на основании описания.
from langchain.tools import tool
from datetime import datetime
# допишем новый тул для агента
@tool("get_current_date", description="Return current date and time for user query")
def get_current_date() -> str:
    """Возвращает текущую дату в формате YYYY-MM-DD"""
    return datetime.now().strftime("%Y-%m-%d")
@tool("string_invert", description="Check query and return right query")
def string_invert(query: str) -> str:
    return query[::-1]



добавим в список тулов инверсию строки

In [27]:
tools = [wikitool, get_current_date, string_invert]

# Агенты на основе маленьких LLM часто галюцинируют по поводу использования
# инструментов и могут подменять результаты работы инструмента своими галлюцинациями.
# Для таких LLM нужно четко прописывать системные промпты.
# перепишем системный промт ещё раз
my_prompt1 = (
    f"Четко следуй инструкциям. Пользовательский запрос может содержать ошибки, его нужно проверять. Для проверки всегда используй {tools[2]}. Используй {tools[0]} для поиска информации.Если вопрос требует:\
- текущей даты или времени — ОБЯЗАТЕЛЬНО используй инструмент get_current_date {tools[1]}. В ответе явно укажи ответ {tools[1]}.")

agent_executor = create_agent(chat_model, tools, system_prompt=my_prompt1)


In [28]:
input_message = {
    "role": "user",
    "content": "Какая сейчас дата?"
}

result = agent_executor.invoke({"messages": [input_message]})

for message in result["messages"]:
  message.pretty_print()

================================ Human Message =================================

Какая сейчас дата?
================================== Ai Message ==================================

<|im_start|>system
Четко следуй инструкциям. Пользовательский запрос может содержать ошибки, его нужно проверять. Для проверки всегда используй name='string_invert' description='Check query and return right query' args_schema=<class 'langchain_core.utils.pydantic.string_invert'> func=<function string_invert at 0x780ed492cea0>. Используй api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/usr/local/lib/python3.12/dist-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=1000) для поиска информации.Если вопрос требует:- текущей даты или времени — ОБЯЗАТЕЛЬНО используй инструмент get_current_date name='get_current_date' description='Return current date and time for user query' args_schema=<class 'langchain_core.utils.pydantic.get

In [ ]:
# Удачи в построении интеллектуальных систем!